# Automatic data downloading

In this example we will show how one can download data from a Jupyter notebook.

The [PhysOcean](https://github.com/gher-uliege/PhysOcean.jl) module provides ways to automatically download data from 
- the [World Ocean Database](https://www.nodc.noaa.gov/OC5/WOD/pr_wod.html) and
- the [CMEMS](http://marine.copernicus.eu/) In-Situ TAC.

This module can be installed by: 

```julia
using Pkg
Pkg.add("PhysOcean")

```
or in the package REPL
```julia
(@v1.11) pkg> add PhysOcean
```
We make sure that the latest version is installed.

In [ ]:
using Pkg
Pkg.add(PackageSpec(name="PhysOcean", rev="master"))

In [3]:
using Makie, CairoMakie, GeoMakie
using PhysOcean           # Download data from the World Ocean Database and Copernicus
using DIVAnd              # DIVAnd 
using Dates
using Statistics
include("../config.jl")

"https://dox.uliege.be/index.php/s/qJtEotmkCZVqcx8/download"

## Settings
Define the time and geospatial bounding box for downloading the data

In [ ]:
# resolution (the resolution is only used for DIVAnd analyses)
dx = dy = 0.25   # medium size test 

# vectors defining the longitude and latitudes grids
# Here longitude and latitude correspond to the Mediterranean Sea
lonr = -7:dx:37
latr = 30:dy:46

# time range of the in-situ data
timerange = [Date(2016, 1, 1), Date(2016, 12, 31)]

In [ ]:
# Name of the variable
varname = "Salinity"

<div class="alert alert-block alert-warning">
⚠️ Please use your own email address (!)<br>
It is only use to get notified by mail once the dataset is ready.
</div>

In [ ]:
if isfile("email.txt")
    email = strip(read("email.txt", String))
    @info("Getting email address from email.txt")
else
    @warn("Create a file 'email.txt' if you want to query data to the World Ocean Database")
end

## Download the data

An example for bulk access data from the World Ocean Database (WOD), simulating a web-user.    
Here you have to possibilities:
1. Execute the next cell to query data from the WOD. It can take several tens of minutes.
2. Use an archive already extracted from the WOD, so there is no need to wait <br>
(cell starting with `download_check(...)`).

Next define the directory where the results will be saved. This directory must exist and must be empty.   
The command `mkpath` will create this path (including parent path).

In [ ]:
basedir = joinpath(datadir, "WOD-temporary-dir")
isdir(basedir) ? rm(basedir, recursive=true) : mkpath(basedir)
WorldOceanDatabase.download(lonr, latr, timerange, varname, email, basedir);

__2nd possibility:__ download and extract archive containing a sample of WOD data.

In [ ]:
download_check(WODdatafile, WODdatafileURL)
extractcommand = `tar -C $(datadir) -xzf $(WODdatafile)`
run(extractcommand);

## Load data
Load the data into memory and perform, if needed, an additional subsetting

In [ ]:
# load all data under basedir as a double-precision floating point variable
obsval, obslon, obslat, obsdepth, obstime, obsid =
    WorldOceanDatabase.load(Float64, basedir, "Salinity");
@info("Number of data points: $(length(obsval))")

Check some observation IDs

In [ ]:
@show obsid[1], obsid[2];

With `checkobs` we get an overview of the extremal values of each dimension and variable.

In [ ]:
checkobs((obslon, obslat, obsdepth, obstime), obsval, obsid)

## Additional sub-setting 
Based on time and depth for plotting.     
For instance the month can be extracted from the `Date` using `Dates.month.(obstime)`.

In [ ]:
# depth range levels
depthr = [0.0, 20.0]

# month range (January to March)
timer = [1, 3]

# additional sub-setting and discard wrong negative salinities
sel = (
    (obsval .> 0) .&
    (minimum(depthr) .<= obsdepth .<= maximum(depthr)) .&
    (minimum(timer) .<= Dates.month.(obstime) .<= maximum(timer))
);

@show typeof(sel);
@show size(sel);

The new variables (ending by `sel`) are a sub-selection based on the previous criteria.

In [ ]:
valsel = obsval[sel]
lonsel = obslon[sel]
latsel = obslat[sel]
depthsel = obsdepth[sel]
timesel = obstime[sel]
idssel = obsid[sel];

Let's perform again the check.

In [ ]:
checkobs((lonsel, latsel, depthsel, timesel), valsel, idssel)

Number of selected data points

In [ ]:
length(valsel)

## Bathymetry download 
For plotting purpose. See [06-topography](06-topography.ipynb) for details.

In [ ]:
bathname = gebco16file
download_check(gebco16file, gebco16fileURL)
bathisglobal = true

# Extract the bathymetry for plotting
bx, by, b = DIVAnd.extract_bath(bathname, bathisglobal, lonr, latr);

Create a simple plot to show the domain.

In [ ]:
plot_bathy(bx, by, b, xticks = -7.0:4.0:37, yticks = 30.0:3.0:48.0)

## Data plotting
The bathymetry is used to display a land-sea mask using the `contourf` function with 2 levels.      
The data are shown as colored circles using `scatter`.

In [ ]:
f = Figure()
ax = GeoAxis(f[1, 1], title = "Observations")
contourf!(ax, bx, by, b, levels = [-1e5, 0, 1e5], colormap = Reverse("binary"))
sc = scatter!(ax, obslon[sel], obslat[sel]; color = obsval[sel], markersize = 3)
Colorbar(f[2, 1], sc, vertical = false)
f

# Check for duplicates

There are two ways to call the function `checkduplicates`:

In [ ]:
?DIVAnd.Quadtrees.checkduplicates

We load a small ODV file containing data in the same domain to test the duplicate detection.     
We use the function `ODVspreadsheet.load` available within `DIVAnd.jl`.

In [ ]:
download_check(smallODVsamplefile, smallODVsamplefileURL)

In [ ]:
obsval_ODV, obslon_ODV, obslat_ODV, obsdepth_ODV, obstime_ODV, obsid_ODV =
    ODVspreadsheet.load(
        Float64,
        [smallODVfile],
        ["Water body salinity"];
        nametype = :localname,
    );

In [ ]:
length(obsval_ODV)

In [ ]:
checkobs((obslon_ODV, obslat_ODV, obsdepth_ODV, obstime_ODV), obsval_ODV, obsid_ODV)

In [ ]:
f = Figure()
ax = GeoAxis(f[1, 1], title = "Observations from ODV file")
contourf!(ax, bx, by, b, levels = [-1e5, 0, 1e5], colormap = Reverse("binary"))
scatter!(ax, obslon_ODV, obslat_ODV, color = obsval_ODV, markersize = 3)
Colorbar(f[2, 1], sc, vertical = false)
f

Look for duplicates
* within 0.01 degree (about 1km)
* within 0.01 m depth
* within 1 minute.      

Difference in value is 0.01 psu.

In [ ]:
dupl = DIVAnd.Quadtrees.checkduplicates(
    (obslon_ODV, obslat_ODV, obsdepth_ODV, obstime_ODV),
    obsval_ODV,
    (obslon, obslat, obsdepth, obstime),
    obsval,
    (0.01, 0.01, 0.01, 1 / (24 * 60)),
    0.01,
);

In [ ]:
size(obsval) == size(dupl)

* `dupl` is an array of the same length as `obsval`
* If the i-th element of `dupl` is an empty list, then the i-th element in `obsval` is probably not a duplicate
* Otherwise, the i-th element in `obsval` is probably a duplicate of the element `val_ODV` with the indices `dupl[i]`.

In [ ]:
dupl[1]

To get a list of possible duplicates, we check for the elements of `dupl` that are not empty.

In [ ]:
index = findall(.!isempty.(dupl))

Number of duplicate candidates

In [ ]:
length(index)

Check the first reported duplicate

In [ ]:
if length(index) > 0
    index_WOD = index[1]
else
    @info("No duplicate detected")
end

Show its coordinates and value from the ODV file:

In [ ]:
obslon[index_WOD],
obslat[index_WOD],
obsdepth[index_WOD],
obstime[index_WOD],
obsval[index_WOD]

They are quite close to the data point with the index:

In [ ]:
dupl[index_WOD]

In [ ]:
index_ODV = dupl[index_WOD][1]

In [ ]:
obslon_ODV[index_ODV],
obslat_ODV[index_ODV],
obsdepth_ODV[index_ODV],
obstime_ODV[index_ODV],
obsval_ODV[index_ODV]

Indeed, it is quite likely that they are duplicates.

Combine the dataset and retain only new points from WOD

In [ ]:
newpoints = findall(isempty.(dupl));
@show length(newpoints)

In [ ]:
obslon_combined = [obslon_ODV; obslon[newpoints]];
obslat_combined = [obslat_ODV; obslat[newpoints]];
obsdepth_combined = [obsdepth_ODV; obsdepth[newpoints]];
obstime_combined = [obstime_ODV; obstime[newpoints]];
obsval_combined = [obsval_ODV; obsval[newpoints]];
obsids_combined = [obsid_ODV; obsid[newpoints]];

## CMEMS data download

<div class="alert alert-block alert-warning">
⚠️ A function <code>CMEMS.download</code> was implemented, but due to the change in CMEMS data access, it is not working anymore.
</div>